In [1]:
#
# Import Packages Required for this Notebook
#
from pyspark.sql import SparkSession
import requests
import json
import msal
import time

print("Successfully imported all packages for this notebook.")

StatementMeta(, 37edba37-cf45-4ffb-a8f5-366be6ef3df0, 3, Finished, Available, Finished)

Successfully imported all packages for this notebook.


In [3]:
#
# Get required secrets from the key vault
#
vault_uri = "https://kv-fabric-dev-eastus2.vault.azure.net/"

# Retrieve secret from Key Vault using mssparkutils
TENANT_ID = mssparkutils.credentials.getSecret(vault_uri, "TENANT-ID")
CLIENT_ID = mssparkutils.credentials.getSecret(vault_uri, "CLIENT-ID")
CLIENT_SECRET = mssparkutils.credentials.getSecret(vault_uri, "CLIENT-SECRET")

# Use the secret securely without printing
print("Secrets retrieved successfully (not displayed for security reasons).")

StatementMeta(, 37edba37-cf45-4ffb-a8f5-366be6ef3df0, 5, Finished, Available, Finished)

Secret retrieved successfully (not displayed for security reasons).


In [ ]:
#
# Verify that key vault items cannot be viewed in clear text
#
print(f"The value of the tenant ID is {TENANT_ID}")
print(f"The value of the client ID is {CLIENT_ID}")
print(f"The value of the client secret is {CLIENT_SECRET}")

StatementMeta(, 37edba37-cf45-4ffb-a8f5-366be6ef3df0, 21, Finished, Available, Finished)

The value of the tenant ID is [REDACTED]
The value of the client ID is [REDACTED]
The value of the client secret is [REDACTED]


In [4]:
#
# Configure run-time parameters for this notebook
#
WORKSPACE_ID = "3ac7ce42-ae74-4e7d-8ac3-5ce8358a30df"
DATASET_ID = "6e5062c5-e912-48ca-93a9-a40c28a855ac"

print("Successfully configured all paramaters for this run.")

StatementMeta(, 37edba37-cf45-4ffb-a8f5-366be6ef3df0, 6, Finished, Available, Finished)

Successfully configured all paramaters for this run.


In [5]:
#
# Connect to Fabric and get the authorization token
#

# Azure AD Details
scope = ["https://analysis.windows.net/powerbi/api/.default"]  # Fabric API scope
authority = f"https://login.microsoftonline.com/{TENANT_ID}"

# Create an MSAL application instance
app = msal.ConfidentialClientApplication(CLIENT_ID, CLIENT_SECRET, authority)

# Get the Access Token
token_response = app.acquire_token_for_client(scopes=scope)

# Check if the token was retrieved successfully
if "access_token" in token_response:
    access_token = token_response["access_token"]
    print("✅ Access Token Retrieved Successfully!")
else:
    print(f"❌ Error getting access token: {token_response.get('error_description')}")

StatementMeta(, 37edba37-cf45-4ffb-a8f5-366be6ef3df0, 7, Finished, Available, Finished)

✅ Access Token Retrieved Successfully!


In [17]:
#
# Refresh the Warehouse semantic model (dataset) in the GOLD layer
#

# API Headers with Access Token
headers = {
    "Authorization": f"Bearer {access_token}",
    "Content-Type": "application/json"
}

# Power BI API URL
url = f"https://api.powerbi.com/v1.0/myorg/groups/{WORKSPACE_ID}/datasets/{DATASET_ID}/refreshes"

# Execute the Power BI REST API
response = requests.post(url, headers=headers)
if response.status_code == 202:
    print("✅ Refresh successfully submitted for the Fabric Semantic Model!")
else:
    print("❌ Error:", response.text)

StatementMeta(, 37edba37-cf45-4ffb-a8f5-366be6ef3df0, 19, Finished, Available, Finished)

✅ Refresh successfully submitted for the Fabric Semantic Model!


In [18]:
#
# Check the Warehouse semantic model (dataset) refresh
#

# API Headers with Access Token
headers = {
    "Authorization": f"Bearer {access_token}",
    "Content-Type": "application/json"
}

# Power BI API URL
url = f"https://api.powerbi.com/v1.0/myorg/groups/{WORKSPACE_ID}/datasets/{DATASET_ID}/refreshes"

# Function to check the refresh status and return when done
def check_refresh_status():
    """Polls the API until the dataset refresh is 'Completed' or 'Failed'."""
    while True:
        response = requests.get(url, headers=headers)

        if response.status_code == 202:
            print("🔄 Refresh request accepted. Waiting before polling again...")
            time.sleep(10)
            continue  # Skip the rest and retry

        if response.status_code == 200:
            try:
                response_json = response.json()
                refreshes = response_json.get("value", [])
                if refreshes:
                    latest_refresh = refreshes[0]  # Get the most recent refresh
                    status = latest_refresh.get("status")
                    print(f"🟢 Current status: {status}")

                    if status == "Completed":
                        print("✅ Dataset refresh completed successfully!")
                        break
                    elif status in ["Failed"]:
                        print("❌ Dataset refresh failed or encountered an issue.")
                        break
                    else:
                        print("⚠️ Refresh in-progress. Sleeping...")
                else:
                    print("⚠️ No refresh history found.")
                    break
            except json.JSONDecodeError as e:
                print(f"❌ JSON decoding error: {e}. Raw response: {response.text}")
                break
        else:
            print(f"❌ Error fetching refresh status: {response.status_code} - {response.text}")
            break

        time.sleep(10)  # Wait 10 seconds before checking again

# Start polling
check_refresh_status()

StatementMeta(, 37edba37-cf45-4ffb-a8f5-366be6ef3df0, 20, Finished, Available, Finished)

🟢 Current status: Unknown
⚠️ Refresh in-progress. Sleeping...
🟢 Current status: Unknown
⚠️ Refresh in-progress. Sleeping...
🟢 Current status: Completed
✅ Dataset refresh completed successfully!
